In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
cls_token = np.load('../outputs/development_fmri_brainlm_a424_650M/zero-shot/650M_cls_token_brainlm.npy')
phenotype = pd.read_csv("../data/external/development_fmri/development_fmri/participants.tsv", index_col='participant_id', sep='\t')
y_ageclass = phenotype['Child_Adult']

In [14]:
cls_token.squeeze()

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]],
      shape=(155, 241), dtype=float16)

In [ ]:
pca = PCA(n_components=100)
pca.fit(cls_token)

plt.figure(figsize=(8, 4))
pc_index = np.arange(50) + 1
plt.bar(pc_index, pca.explained_variance_ratio_[:50], label="Explained Variance Per Component")
plt.plot(pc_index, pca.explained_variance_ratio_[:50].cumsum(), 'o-', 
         linewidth=2, color='red', label="Cumulative Variance Explained")
plt.title('Explained Variance for Principal Components of\nReduced CLS Tokens', fontsize=14)
plt.xlabel('Principal Component', fontsize=14)
plt.ylabel('% Variance Explained', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14)
# plt.savefig("inference_plots/pca_scree_plot_cls_tokens_50components.png", bbox_inches="tight", facecolor="white")
plt.show()
# plt.close()

ValueError: Found array with dim 3. PCA expected <= 2.

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X_features, 
    y_ageclass, 
    test_size = 0.2, 
    shuffle = True, 
    stratify = y_ageclass, 
    random_state = 42
)

# print the size of our training and test groups
print('training:', len(X_train), 'testing:', len(X_test))
